## Preprocessing

In [2]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Tensorflow
import tensorflow as tf

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
#  Import and read the charity_data.csv.
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cols_to_drop = ["EIN"]
df.drop(columns=cols_to_drop, inplace=True)
df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
df.nunique()

,0
NAME,19568
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747


In [6]:
name_count = df['NAME'].value_counts()
name_count[name_count>5]

,count
NAME,
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,408
...,...
OLD OAK CLIFF CONSERVATION LEAGUE INC,6
AMERICAN NEPHROLOGY NURSES ASSOCIATION,6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC,6


In [7]:
names_to_replace = list(name_count[name_count <= 5].index)

df['NAME'] = df['NAME'].where(~df['NAME'].isin(names_to_replace), "Other")

df['NAME'].value_counts()

,count
NAME,
Other,20043
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
...,...
HABITAT FOR HUMANITY INTERNATIONAL,6
DAMAGE PREVENTION COUNCIL OF TEXAS,6
FLEET RESERVE ASSOCIATION,6


In [8]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
df["APPLICATION_TYPE"].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
threshold = 500
app_type_counts = df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = app_type_counts.loc[app_type_counts < threshold].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [10]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
df["CLASSIFICATION"].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = df["CLASSIFICATION"].value_counts()
class_counts.loc[class_counts > 1]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_counts = df["CLASSIFICATION"].value_counts()
classifications_to_replace = class_counts.loc[class_counts < 200].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
Other,1197
C7000,777
C1700,287


In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
df2 = pd.get_dummies(df, dtype=int)
df2.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALPHA PHI OMEGA,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALPHA SIGMA PHI FRATERNITY INC,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ART THERAPY ASSOCIATION INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN INC,NAME_AMERICAN CHEMICAL SOCIETY,NAME_AMERICAN CHRISTIAN FICTION WRITERS INC,NAME_AMERICAN COLLEGE OF HEALTHCARE EXECUTIVES,NAME_AMERICAN COLLEGE OF NURSE-MIDWIVES,NAME_AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES,NAME_AMERICAN FEDERATION OF LABOR & CONGRESS OF INDUSTRIAL ORGS,NAME_AMERICAN FEDERATION OF LABOR AND CONGRESS OF INDUSTRIAL ORGS,NAME_AMERICAN FEDERATION OF STATE COUNTY & MUNICIPAL EMPLOYEES,NAME_AMERICAN HIBISCUS,NAME_AMERICAN INDEPENDENT BUSINESS ALLIANCE,NAME_AMERICAN INSTITUTE OF GRAPHIC ARTS,NAME_AMERICAN IRIS SOCIETY,NAME_AMERICAN LEGION,NAME_AMERICAN LEGION AUXILIARY,NAME_AMERICAN NEPHROLOGY NURSES ASSOCIATION,NAME_AMERICAN POSTAL WORKERS UNION,NAME_AMERICAN SOCIETY OF ADDICTION MEDICINE INC,NAME_AMERICAN STUDENT DENTAL ASSOCIATION,NAME_AMERICAN VOLKSSPORT ASSOCIATION INC,NAME_AMERICAN WATER WORKS ASSOCIATION,NAME_AMERICAN YOUTH FOOTBALL INC,NAME_AMERICANS UNITED FOR SEPARATION OF CHURCH AND STATE,NAME_AMIGOS DE LAS AMERICAS,NAME_AMVETS,NAME_ANCIENT AND FREE ACCEPTED MASONS OF MISSOURI,NAME_ANCIENT FREE & ACCEPTED MASONS OF KANSAS,NAME_ARIZONA FEDERATION OF GARDEN CLUBS,NAME_ASIAN PACIFIC AMERICAN INTERNAL REVENUE EMPLOYEES,NAME_ASSOCIATES OF VIETNAM VETERANS OF AMERICA INC,NAME_ASSOCIATION FOR PROFESSIONALS IN INFECTION CTRL & EPIDEMIOLOGY INC,NAME_ASSOCIATION FOR VASCULAR ACCESS,NAME_ASSOCIATION OF CIVILIAN TECHNICIANS INC,NAME_ASSOCIATION OF CLINICAL RESEARCH PROFESSIONALS,NAME_ASSOCIATION OF OPERATING ROOM NURSES INC,NAME_ASSOCIATION OF PROFESSIONAL GENEALOGISTS,NAME_ASSOCIATION OF SCHOOL BOOSTER CLUBS,NAME_ASSOCIATION OF TRADITIONAL HOOKING ARTISTS,NAME_BABYWEARING INTERNATIONAL INC,NAME_BARNABAS GROUP INC,NAME_BEAVERHEAD COUNTY 4-H FOUNDATION,NAME_BEREAVED PARENTS OF THE USA,NAME_BETA BETA BETA BIOLOGICAL SOCIETY,NAME_BETTER WORLD FUND,NAME_BLUE KNIGHTS MOTORCYCLE CLUB,NAME_BOY SCOUTS OF AMERICA,NAME_BOYS TEAM CHARITY INC,NAME_CAJUN FRENCH MUSIC ASSOCIATION OF LOUISIANA INC,NAME_CALIFORNIA ASSOCIATION FOR THE EDUCATION OF YOUNG CHILDREN,NAME_CALIFORNIA FEDERATION OF WOMENS CLUBS,NAME_CALIFORNIA GARDEN CLUBS INC,NAME_CALIFORNIA HIGHWAY PATROL EXPLORER PROGRAM,NAME_CALIFORNIA STATE CHAPER WOMENS COUNCIL OF REALTORS,NAME_CALIFORNIA STATE UNIVERSITY EMPLOYEES UNION,NAME_CALIFORNIA TEACHERS ASSOCIATION,NAME_CALIFORNIA WOMENS CHORUS,NAME_CASCADE 4-H FOUNDATION,NAME_CBMC INC,NAME_CENTERS FOR SPIRITUAL LIVING,NAME_CHEVRON RETIREES ASSOCIATION,NAME_CHIEF WARRANT & WARRANT OFFICERS ASSOCIATION U S COAST GUARD,NAME_CHRISTIAN MEDICAL & DENTAL SOCIETY,NAME_CIRCLE K INTERNATIONAL INC,NAME_CISD EDUCATIONAL SUPPORT GROUPS INC,NAME_CIVITAN INTERNATIONAL,NAME_CLEAR CREEK ISD GROUPS,NAME_CLEARCREEK I S D SUPPORT GROUPS,NAME_COLLEGE AND UNIVERSITY PROFESSIONAL ASSOCIATION FOR HUMAN RESOURCES,NAME_COMMUNICATION WORKERS OF AMERICA,NAME_CONCERNS OF POLICE SURVIVORS,NAME_CONSTRUCTION OWNERS ASSOCIATION OF AMERICA INC,NAME_CREATIVE OPPORTUNITIES UNLIMITED,NAME_DAMAGE PREVENTION COUNCIL OF TEXAS,NAME_DAUGHTERS OF CHARITY HEALTH SYSTEM,NAME_DAUGHTERS OF ESTHER FELLOWSHIP INTERNATIONAL MINISTRIES INC,NAME_DAUGHTERS OF UNION VETERANS OF THE CIVIL WAR 1861-1865,NAME_DELTA KAPPA GAMMA SOCIETY INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_DEPARTMENT OF ALABAMA AUXILIARY TO THE VFW OF THE U S,NAME_DEPARTMENT OF ALABAMA VETERANS OF FOREIGN WARS OF

In [14]:
# Split our preprocessed data into our features and target arrays
X = df2.drop(columns=["IS_SUCCESSFUL"])
y = df2.IS_SUCCESSFUL

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y) # stratify=True maintains target class percentages
print(X_train.shape, X_test.shape)

(25724, 400) (8575, 400)


In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the model - deep neural net
number_input_features = len(X_train.iloc[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_model = tf.keras.models.Sequential()

# Input layer
nn_model.add(layers.InputLayer(input_shape=(number_input_features,)))

# First hidden layer
nn_model.add(layers.Dense(hidden_nodes_layer1, activation='relu'))

# Dropout layer to prevent overfitting
nn_model.add(layers.Dropout(0.3))

# Hidden Layer 2
nn_model.add(layers.Dense(hidden_nodes_layer2, activation='relu'))

# Dropout layer
nn_model.add(layers.Dropout(0.3))

# Hidden Layer 3
nn_model.add(layers.Dense(hidden_nodes_layer3, activation='relu'))

# Output layer
nn_model.add(layers.Dense(1, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 50)                  │          20,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           1,530 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 30)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             310 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,901 (85.55 KB)

 Trainable params: 21,901 (85.55 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7917 - loss: 0.4309
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7965 - loss: 0.4224
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7959 - loss: 0.4239
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7943 - loss: 0.4264
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7915 - loss: 0.4267
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8016 - loss: 0.4197
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8002 - loss: 0.4204
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7985 - loss: 0.4205
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7996 - loss: 0.4224
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7982 - loss: 0.4181
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8001 - loss: 0.4201
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7900 - loss: 0.4458
Loss: 0.445770800113678, Accuracy: 0.789970874786377


In [24]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity.h5")

In [60]:
from tensorflow.keras import layers, callbacks

# Define the model with increased complexity
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32

nn_model_optimized = tf.keras.models.Sequential()

# Input layer
nn_model_optimized.add(layers.InputLayer(input_shape=(number_input_features,)))

# First hidden layer with Batch Normalization
nn_model_optimized.add(layers.Dense(hidden_nodes_layer1, activation='relu'))
nn_model_optimized.add(layers.BatchNormalization())
nn_model_optimized.add(layers.Dropout(0.2))

# Hidden Layer 2 with Batch Normalization
nn_model_optimized.add(layers.Dense(hidden_nodes_layer2, activation='relu'))
nn_model_optimized.add(layers.BatchNormalization())
nn_model_optimized.add(layers.Dropout(0.2))

# Hidden Layer 3 with Batch Normalization
nn_model_optimized.add(layers.Dense(hidden_nodes_layer3, activation='relu'))
nn_model_optimized.add(layers.BatchNormalization())

# Output layer
nn_model_optimized.add(layers.Dense(1, activation='sigmoid'))

# Check the structure of the model
nn_model_optimized.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                     │ (None, 128)                 │          51,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 62,593 (244.50 KB)

 Trainable params: 62,145 (242.75 KB)

 Non-trainable params: 448 (1.75 KB)

In [61]:
# Compile the model
nn_model_optimized.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [62]:
# Implement Early Stopping
early_stopping_monitor = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [63]:
# Implement Learning Rate Scheduler
lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

In [64]:
# Fit the model with callbacks
history = nn_model_optimized.fit(
    X_train_scaled,
    y_train,
    epochs=250,
    batch_size=84,
    validation_data=(X, y),
    callbacks=[early_stopping_monitor, lr_scheduler]
)

Epoch 1/250
307/307 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7019 - loss: 0.5847 - val_accuracy: 0.5324 - val_loss: 36637.6289 - learning_rate: 0.0010
Epoch 2/250
307/307 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7838 - loss: 0.4624 - val_accuracy: 0.5324 - val_loss: 24894.3750 - learning_rate: 0.0010
Epoch 3/250
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7895 - loss: 0.4447 - val_accuracy: 0.5324 - val_loss: 6784.8271 - learning_rate: 0.0010
Epoch 4/250
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7961 - loss: 0.4354 - val_accuracy: 0.5324 - val_loss: 12029.6553 - learning_rate: 0.0010
Epoch 5/250
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7888 - loss: 0.4395 - val_accuracy: 0.5324 - val_loss: 4980.2383 - learning_rate: 0.0010
Epoch 6/250
307/307 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7943 - loss: 0.4321 - val_accuracy: 0.4676 - val_loss: 422.4406 - learning_rate: 0.0010
Epoch 7/250
307/307 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accura

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_optimized.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7885 - loss: 0.4361
Loss: 0.43614354729652405, Accuracy: 0.7884548306465149


In [66]:
# Export our model to HDF5 file
nn_model_optimized.save("AlphabetSoupCharity_Optimization.h5")

In [ ]:
#Optimization achieved with help from Xpert :)